# Agent Router – Integrasi Common Information Tool
Notebook ini menambahkan **tool** `common_information_tool` ke Agent Haystack, serta *routing* otomatis antara pertanyaan **produk** (Product Recommendation) dan **informasi umum** (Common Information).

In [ ]:
# --- Setup & Imports
import os
import dotenv
from typing import Annotated
from functools import partial

from haystack import Pipeline, component
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.utils import Secret
from haystack_experimental.chat_message_stores.in_memory import InMemoryChatMessageStore
from haystack_experimental.components.writers import ChatMessageWriter
from haystack.tools.tool import Tool

# Import pipeline Common Info dari notebook RAG sebelumnya (atau copy kelasnya)
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder

dotenv.load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
MONGO_CONNECTION_STRING = os.environ.get("MONGO_CONNECTION_STRING")
assert OPENAI_API_KEY and MONGO_CONNECTION_STRING, "Set OPENAI_API_KEY dan MONGO_CONNECTION_STRING"

DB_NAME = "depato_store"
COLLECTION_CI = "common_information"
VECTOR_INDEX_CI = "vector_index_common_info"
FTS_INDEX_CI = "search_index_common_info"

ci_store = MongoDBAtlasDocumentStore(
    database_name=DB_NAME,
    collection_name=COLLECTION_CI,
    vector_search_index=VECTOR_INDEX_CI,
    full_text_search_index=FTS_INDEX_CI,
)

from haystack.components.builders import ChatPromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator

class CommonInfoToolPipeline:
    def __init__(self, store):
        from haystack import Pipeline
        self.pipeline = Pipeline()
        self.pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
        self.pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(document_store=store, top_k=6))
        self.pipeline.add_component("prompt_builder", ChatPromptBuilder(variables=["query","documents"], required_variables=["query","documents"]))
        self.pipeline.add_component("generator", OpenAIChatGenerator(model="gpt-4.1-2025-04-14", api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])))

        self.pipeline.connect("embedder", "retriever")
        self.pipeline.connect("retriever", "prompt_builder.documents")
        self.pipeline.connect("prompt_builder.prompt", "generator.messages")

    def run(self, query: str) -> str:
        msgs = [
            ChatMessage.from_system("Kamu adalah asisten yang menjawab pertanyaan umum (pembelian/pengiriman/refund). Jawab dari dokumen yang ada."),
            ChatMessage.from_user(
                """
                Pertanyaan: {{query}}
                {% for d in documents %}
                ---
                {{ d.content }}
                {% endfor %}
                Jawab ringkas dan beri langkah-langkah bila ada.
                """
            )
        ]
        out = self.pipeline.run(
            data={
                "embedder": {"text": query},
                "prompt_builder": {"query": query, "template": msgs},
            },
            include_outputs_from=["generator"]
        )
        return out["generator"]["replies"][0].text

common_info_pipeline = CommonInfoToolPipeline(ci_store)

In [ ]:
def answer_common_information(query):
    return common_info_pipeline.run(query)

common_information_tool = Tool(
    name="common_information_tool",
    description="Jawab pertanyaan umum terkait proses jual-beli (pembelian, pengiriman, refund, retur, tracking, metode bayar).",
    function=answer_common_information,
    parameters={
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "Pertanyaan user"}
        },
        "required": ["query"]
    }
)

In [ ]:
retrieve_and_generate_tool = None  # ganti dengan tool asli Anda bila ada

In [ ]:
chat_store = InMemoryChatMessageStore()
writer = ChatMessageWriter(chat_store)

tools = [common_information_tool] + ([retrieve_and_generate_tool] if retrieve_and_generate_tool else [])
tools = [t for t in tools if t is not None]

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4.1-2025-04-14"),
    tools=tools,
    system_prompt="""
    Kamu adalah Shop Assistant.
    ROUTING:
    - Jika pertanyaan tentang pembelian, pembayaran, pengiriman, tracking, retur, refund → gunakan common_information_tool.
    - Jika pertanyaan tentang rekomendasi produk/material/harga/kategori → gunakan tool rekomendasi produk.
    - Jika tidak relevan → jelaskan keterbatasan dengan sopan.
    WORKFLOW:
    Analisis niat user, pilih tool bila perlu. Jika cukup dijawab tanpa tool, jawab langsung.
    """,
    exit_conditions=["text"],
    max_agent_steps=10,
)
agent.warm_up()
print("Agent siap dengan", [t.name for t in tools])

In [ ]:
messages = [ChatMessage.from_user("Bagaimana cara mengajukan refund?")]
resp = agent.run(messages=messages)
print(resp["messages"][-1].text)